# Chapter 2: Working with Text Data

- I'm learning how to prepare and clean text for language modeling
- My goal: understand tokenization and build a small vocabulary
- I want to turn raw text into numeric inputs I can train on

Packages used in this notebook:

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

## 2.1 Understanding word embeddings

- I'm learning that for LLMs, tokens are represented as vectors in a high-dimensional space
- These vectors encode relationships that the model can learn
- I find it helpful to use 2D sketches to build intuition about this much larger space

## 2.2 Tokenizing text

- Tokenization splits text into units (words, punctuation, or subwords)
- These units become the inputs the model will work with
- I’ll use a public-domain text (The Verdict) as sample data

In [ ]:
import os

# Check for local file first (zero-copy policy compliance)
# The file should already exist in the notebook directory
file_path = "the-verdict.txt"

if not os.path.exists(file_path):
    # Fallback: download from public domain source if local file missing
    # Note: This is a public domain text by Edith Wharton
    import requests
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {file_path} from external source")
else:
    print(f"Using local file: {file_path}")


In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

- Goal: turn the raw text into tokens a model can consume
- I’ll build a simple regex tokenizer first, then scale it to the full text
- Start by splitting on whitespace to see the baseline behavior

In [ ]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

- I noticed that whitespace-only splitting misses punctuation boundaries
- My next step: treat commas and periods as their own tokens

In [ ]:
result = re.split(r'([,.]|\s)', text)

print(result)

- I noticed the split introduces empty strings
- I'll filter them out to keep only meaningful tokens

In [ ]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

- I'm expanding the pattern to cover more punctuation (e.g., ?, !, quotes)
- This should produce cleaner, more realistic token streams for my model

In [ ]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

- The regex looks good enough for a first pass
- Now I'll apply it to the full text I loaded earlier

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

- Let me count the total number of tokens I produced

In [ ]:
print(len(preprocessed))

## 2.3 Converting tokens into IDs

- I'm learning that models need numbers, not strings
- I'll build a vocabulary mapping each unique token to an integer ID
- Starting by listing unique tokens in a consistent order

In [ ]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)

print(vocab_size)

- I'll create the token → ID dictionary using a simple enumeration

In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- Let me peek at a handful of entries to sanity-check my mapping

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

- Next, I’ll wrap the logic into a small tokenizer class
- Then I’ll test it on a short sample string

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` converts text into token IDs
- `decode` converts token IDs back into text
- These IDs are what we’ll later feed into embeddings for the model

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

- I'll decode the IDs to verify the round-trip works

In [ ]:
tokenizer.decode(ids)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

## 2.4 Adding special context tokens

- I noticed a problem: when I try to tokenize text with words not in my vocabulary, it breaks
- I need a way to handle unknown words and mark text boundaries
- I'm learning about special tokens that can help with this

My understanding so far:
- `[UNK]` can represent words not in vocabulary (handles unknown words)
- `[EOS]` marks end of text (useful when concatenating multiple texts)
- `[PAD]` is for padding shorter texts to match longer ones in batches
- `[BOS]` marks beginning of sequence (some tokenizers use this)

I'm curious about GPT-2's approach:
- GPT-2 uses `<|endoftext|>` instead of all those tokens - much simpler!
- It's like `[EOS]` but also used for padding (since we mask padded tokens anyway)
- GPT-2 doesn't use `[UNK]` because it uses BPE (byte-pair encoding) which breaks words into subword units
- I'll learn about BPE later, but for now I'll add `<|unk|>` and `<|endoftext|>` to handle these cases

My takeaway: GPT-2's simpler approach makes sense - one token does multiple jobs when you use masking properly.

Let me see what happens when I try to tokenize text with words not in my vocabulary:


In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

try:
    tokenizer.encode(text)
except KeyError as e:
    print(f"KeyError: The token {e} is not in the vocabulary.")

- I got an error because "Hello" isn't in my vocabulary
- To handle this, I'll add special tokens like `"<|unk|>"` to represent unknown words
- Since I'm extending the vocabulary anyway, I'll also add `"<|endoftext|>"` which GPT-2 uses to mark the end of text (and between concatenated texts when training on multiple articles or books)

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
len(vocab.items())

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

- I also need to adjust my tokenizer so it knows when and how to use the new `<|unk|>` token

In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

Now let me test the modified tokenizer with some text that includes unknown words and multiple sentences:

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

In [ ]:
tokenizer.encode(text)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
tokenizer.decode(ids)
tokenizer.decode(tokenizer.encode(text))

## 2.5 BytePair encoding

- GPT-2 used BytePair encoding (BPE) as its tokenizer
- it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
- For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
- The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- In this chapter, we are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library, which implements its core algorithms in Rust to improve computational performance
- I created a notebook in the [./bytepair_encoder](../02_bonus_bytepair-encoder) that compares these two implementations side-by-side (tiktoken was about 5x faster on the sample text)

In [ ]:
# pip install tiktoken

In [ ]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

In [ ]:
strings = tokenizer.decode(integers)

print(strings)

## 2.6 Data sampling with a sliding window

- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict:


In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

- For each text chunk, we want the inputs and targets
- Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right

In [ ]:
enc_sample = enc_text[50:]

In [ ]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

- One by one, the prediction would look like as follows:

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

- We will take care of the next-word prediction in a later chapter after we covered the attention mechanism
- For now, we implement a simple data loader that iterates over the input dataset and returns the inputs and targets shifted by one

- Install and import PyTorch (see Appendix A for installation tips)

In [ ]:
import torch
print("PyTorch version:", torch.__version__)

- We use a sliding window approach, changing the position by +1:

In [ ]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4:

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

In [ ]:
second_batch = next(data_iter)
print(second_batch)

- An example using stride equal to the context length (here: 4) as shown below:
- We can also create batched outputs
- Note that we increase the stride here so that we don't have overlaps between the batches, since more overlap could lead to increased overfitting

In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)